DESCRIPTION: SHA-256 Hashing and Bitcoin Mining

# How hashing does bitcoin mining

[Bitcoin](https://en.wikipedia.org/wiki/Bitcoin) uses 2 rounds of SHA-256 hashing to create its blockchains. In this page, we will do rudimentary block chains and hashing to get an idea of how the big picture works.

A "blockchain" is a chain of blocks. Duh! So what is a "block" and what is a "chain"?

As per [what is a block](https://www.investopedia.com/terms/b/block-bitcoin-block.asp), a block is an array of bytes (can be a string) that contains:
* ID of previous block.
* A nonce, or "number used just once" (in this case an integer)
* The size of a block. We'll just make it any size.
* [A Merkle Hash Root](https://en.wikipedia.org/wiki/Merkle_tree)


In [112]:
import hashlib
import datetime
from dataclasses import dataclass

def hash(s):
    '''Return the hash of string @s'''
    m = hashlib.sha256()
    m.update(s.encode("utf-8"))
    m.digest()
    return m.hexdigest()
    
def tprint(s):
    print(f"{s:<10} {hash(s)}")

In [107]:
# Test hash
tprint('hello')
tprint('hellp')
tprint('hell')
tprint('ello')
tprint('Dagny')

# Notice that:
# - the length of the hash is 64 hex characters; i.e. 256 / 4 = 64; e.g. 4-bits per hex character.
# - slight changes in the input string create completely different hashes
# - the same hash is always created for the same string
# - very long strings produce the same length of the hash as short string.
# - Practically impossible to deduce the message from the hash
# - Practically impossible to find two messages that produce the same hash; i.e. a collision.

hello      2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
hellp      fdd7585e08c4e2afd71dcabdb4636c89d557a3f42db9e2040c8bbd1708aa4ce7
hell       0ebdc3317b75839f643387d783535adc360ca01f33c75f7c1e7373adcd675c0b
ello       fcba366c2ebf76bf96fe5a737e4a5350bb54ba224e909c06e97e79e8f5e5ffe5
Dagny      0e87288a44b37942ada3e77dee15273f31910913472e691f815827d8a7dee4d1


In [108]:
# Let's make a very over-simplified block chain, sort of similar to a bitcoin.
# A block will be a string @s and its id. The id is the hash of the string.
# We'll separate fields by a pipe '|'
@dataclass
class Block:
    id: str  # i.e. the hash(body)
    body: str
        
def makeBlock(prevId, nonce, s):
    s = f"{prevId}|{nonce}|{s}"
    id = hash(s)
    return Block(id, s)

def pblock(block):
    '''Print a block nicely'''
    print(f"""
body: "{block.body}"
  id: {block.id}""")
    

In [98]:
# Now let's make a block chain:

blocks = []
blocks.append(makeBlock('', 123, "Once upon a midnight dreary, while I pondered, weak and weary,"))
blocks.append(makeBlock(blocks[0].id, 999, "Over many a quaint and curious volume of forgotten lore"))
blocks.append(makeBlock(blocks[1].id, 999, "While I nodded, nearly napping, suddenly there came a tapping,"))
blocks.append(makeBlock(blocks[2].id, 1, "As of some one gently rapping, rapping at my chamber door."))
blocks.append(makeBlock(blocks[2].id, 2, "As of some one gently rapping, rapping at my chamber door."))
blocks.append(makeBlock(blocks[2].id, 3, "As of some one gently rapping, rapping at my chamber door."))

for block in blocks:
  pblock(block)

print('\nNotice that the last three blocks only differ by the nonce.')



body: "|123|Once upon a midnight dreary, while I pondered, weak and weary,"
  id: 5417fba7f067b2c990c171f822ea8ad7ef24c9b8656941db7421b91e87c33ca9

body: "5417fba7f067b2c990c171f822ea8ad7ef24c9b8656941db7421b91e87c33ca9|999|Over many a quaint and curious volume of forgotten lore"
  id: 82d395f3bc97cafb0c040c56283efb4cf3801e57f9608884c1ca1c8daf47d9c1

body: "82d395f3bc97cafb0c040c56283efb4cf3801e57f9608884c1ca1c8daf47d9c1|999|While I nodded, nearly napping, suddenly there came a tapping,"
  id: f55fbc98fc8b173980634cc034d5136252cca60eeee134d11e97f30df0599bea

body: "f55fbc98fc8b173980634cc034d5136252cca60eeee134d11e97f30df0599bea|1|As of some one gently rapping, rapping at my chamber door."
  id: b07a8665af1c4fd77fd01628f00db8427c09fe42372cce21aa079186371cf25e

body: "f55fbc98fc8b173980634cc034d5136252cca60eeee134d11e97f30df0599bea|2|As of some one gently rapping, rapping at my chamber door."
  id: 13ab7e683f451b1ab7688729dd81de023f3a212ba5f9084279e4894c3b60cd0d

body: "f55fbc98fc8b173

In [99]:
# So what is this nonce thingy?
# Well, mining is the act of finding the smallest hash for a block by varying only the nonce.
# We can't change the prevId or the message, so the nonce is all we can change.
# But what does it mean smallest hash? We can compare hashes like we can any two strings lexically; e.g 
# "cat" comes before "dog", which comes before "doggy"

# Let's sort the blocks above by id:
for block in sorted(blocks, key=lambda x: x.id):
    print(block.id)
    
# You can see that we can compare ids. The first id in the list is less than the next ones that follow.

13ab7e683f451b1ab7688729dd81de023f3a212ba5f9084279e4894c3b60cd0d
5417fba7f067b2c990c171f822ea8ad7ef24c9b8656941db7421b91e87c33ca9
82d395f3bc97cafb0c040c56283efb4cf3801e57f9608884c1ca1c8daf47d9c1
b07a8665af1c4fd77fd01628f00db8427c09fe42372cce21aa079186371cf25e
f03a5476e0f1e07026ac07ff39b6f1961d6c7d8412f7ac79b705bf66ea921160
f55fbc98fc8b173980634cc034d5136252cca60eeee134d11e97f30df0599bea


In [129]:
# Mining is the act of changing the nonce of a block until we get an id that is less than some value,
# called the "mining difficulty". Let's start with a simple difficulty:

difficulty = "00002" # means we will only accept id's that are less than this value. Try adding another '0'

# Let's start mining

def mine(prevId, message):
    '''Find a block id that is less than the difficulty.'''
    nonce = 0
    start = datetime.datetime.now()
    while (True):
        block = makeBlock(prevId, nonce, message)
        #print(f"{nonce:>4} {block}")
        if block.id <= difficulty:
            break
        nonce += 1
    secs = (datetime.datetime.now() - start).total_seconds()
    print(f"\nBlock mined in {nonce} iterations. Took {secs:0.2f} seconds")
    return block
    
block = mine("", "Attack at dawn!")
print(block)

block = mine(block.id, "One if by land!")
print(block)


block = mine(block.id, "Two if by sea!")
print(block)



Block mined in 601953 iterations. Took 2.40 seconds
Block(id='00001528904168ad83b514203a840c6197905acae857737f2766a9d4eacb6d9a', body='|601953|Attack at dawn!')

Block mined in 1567282 iterations. Took 6.32 seconds
Block(id='00001a2c2dbc5fa899b998d839ae0206ce2f992131cd275db5fbe8cfa8436da5', body='00001528904168ad83b514203a840c6197905acae857737f2766a9d4eacb6d9a|1567282|One if by land!')

Block mined in 270536 iterations. Took 1.08 seconds
Block(id='00001e76f866eeb5aa6bbb4483f301d71c7fed803c7b52c884fa1748a15b9165', body='00001a2c2dbc5fa899b998d839ae0206ce2f992131cd275db5fbe8cfa8436da5|270536|Two if by sea!')
